## Toggle Flip-Flop

In this example we create a toggle flip-flop (TFF) from a d-flip-flop and an xor gate. This shows how to construct sequential logic circuits using `Magma`.

In [1]:
import magma as m

import os
os.environ['MANTLE_TARGET'] = 'ice40'
from mantle import DFF

import mantle lattice ice40
import mantle lattice mantle40


In [2]:
def tff():
    # instance a dff to hold the state of the toggle flip-flop - this needs to be done first
    ff = DFF()

    # compute the next state as the not of the old state ff.O
    ff( ~ff.O )

    return ff.O

In [3]:
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.Clock.on()
icestick.J3[0].rename('J3').output().on()

main = icestick.DefineMain()
m.wire( tff(), main.J3 )
m.EndDefine()

## Compile and Build

In [4]:
m.compile("build/tff", main)

compiling main


In [5]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif tff.blif' tff.v
arachne-pnr -q -d 1k -o tff.txt -p tff.pcf tff.blif 
icepack tff.txt tff.bin
# iceprog tff.bin

/Users/hanrahan/git/magmathon/notebooks/tutorial/build


J3 should now be toggling.

Here is the verilog.

In [6]:
with open("build/tff.v", "r") as tff_verilog:
    print(tff_verilog.read())

module main (output  J3, input  CLKIN);
wire  inst0_Q;
wire  inst1_O;
SB_DFF inst0 (.C(CLKIN), .D(inst1_O), .Q(inst0_Q));
SB_LUT4 #(.LUT_INIT(16'h5555)) inst1 (.I0(inst0_Q), .I1(1'b0), .I2(1'b0), .I3(1'b0), .O(inst1_O));
assign J3 = inst0_Q;
endmodule




Note that the TFF uses the ICE40 `SB_DFF` and `SB_LUT4` primitives.